###### Imports and Settings

UPDATED 10/14/2024 THROUGH SEPTEMBER 2024

In [1]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
import sqlite3 as sq

In [2]:
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
#calculate real and percent change between all columns for all possible time frames
def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames

# This notebook outlines the download and formatting process for the Zillow Home Value Index for counties and places in the GNRC operating region.  

Go to this page: https://www.zillow.com/research/data/  

+ Under "HOME VALUES", select Data Type "ZHVI All Homes (SFR, Condo/Co-op) Time Series, Smoothed, Seasonally Adjusted" and download this for Metro & US, State, County, and City.   

### Save these csvs as they come in the Data Downloads folder

In [4]:
hv_metrous = pd.read_csv('../Data Downloads/Zillow_MetroUS_ZHVI.csv')
hv_state = pd.read_csv('../Data Downloads/Zillow_State_ZHVI.csv')
hv_county = pd.read_csv('../Data Downloads/Zillow_County_ZHVI.csv')
hv_place = pd.read_csv('../Data Downloads/Zillow_City_ZHVI.csv')

The county codes are not FIPS codes, unsure what they are but our region contains the following:
Cheatham:2185, Davidson:2243, Dickson:1668 , Houston:1784, Humphreys:2728, Macon:623, Maury:632, Montgomery:2982, Robertson:2834, Rutherford:3016, Sumner:1407, Stewart:2044, Trousdale:2856, Williamson:3080, Wilson:1496, (KY) Allen:369 , (KY) Simpson:2028

In [5]:
#filter down the metro and us files to include only the US and the Nashville and Clarksville MSAs
metrous = [102001, 394902, 394471]
hv_metrous = hv_metrous.loc[hv_metrous['RegionID'].isin(metrous)].reset_index(drop = True)
#filter the one state file
hv_state = hv_state.loc[hv_state['RegionID'] == 53].reset_index(drop = True)
#filter the county files, Simpson Co KY is RegionID 2028, but doesn't have associated data until recent years so not including for now
counties = [2185, 2243, 1668, 1784, 2728, 623, 632, 2982, 2834, 3016, 1407, 2044, 2856, 3080, 1496, 369]
hv_county = hv_county.loc[hv_county['RegionID'].isin(counties)].reset_index(drop = True)
#filter the place files
places = [41932, 30583, 10843, 30993, 49233, 45339, 11564, 32006, 46091, 25534, 42878, 39894, 19523, 6118, 26161, 54450, 7208, 27227, 29482, 41690]
hv_place = hv_place.loc[hv_place['RegionID'].isin(places)].reset_index(drop = True)

## Fix up home value DFs

In [6]:
hv_place = hv_place.drop(columns = ['RegionID', 'SizeRank', 'RegionType', 'StateName', 'State', 'Metro', 'CountyName']).set_index(('RegionName'))
hv_county = hv_county.drop(columns = ['RegionID', 'SizeRank', 'RegionType', 'StateName', 'State', 'Metro', 'StateCodeFIPS', 'MunicipalCodeFIPS']).set_index(('RegionName'))
hv_state = hv_state.drop(columns = ['RegionID', 'SizeRank', 'RegionType', 'StateName']).set_index(('RegionName'))
hv_metrous = hv_metrous.drop(columns = ['RegionID', 'SizeRank', 'RegionType', 'StateName']).set_index(('RegionName'))

In [7]:
#append the home value dataframes
frame = [hv_place, hv_county, hv_state, hv_metrous]
emptyframe = pd.DataFrame()
for df in frame:
    emptyframe = pd.concat([emptyframe, df])    
hvs = emptyframe

In [8]:
#transpose and rename the index to NAME
data = hvs.transpose().reset_index().rename(columns = {'index':'NAME'})

In [9]:
#adjust the geo names using str.split and index into the correct year
year = data['NAME'].str.split(pat = "/", expand = True)
data['Year'] = year[2]

In [10]:
data.drop(columns = 'NAME', inplace = True)

In [11]:
data.head()

RegionName,Nashville,Murfreesboro,Clarksville,Franklin,Hendersonville,Lebanon,Columbia,Mount Juliet,Smyrna,Gallatin,Spring Hill,Brentwood,La Vergne,Springfield,Nolensville,Ashland City,White House,Thompsons Station,Fairview,Davidson County,Rutherford County,Williamson County,Montgomery County,Sumner County,Wilson County,Maury County,Robertson County,Dickson County,Cheatham County,Macon County,Allen County,Humphreys County,Stewart County,Trousdale County,Houston County,Tennessee,United States,"Nashville, TN","Clarksville, TN",Year
0,133499.6876,130722.2402,114793.7813,235806.7460,170639.8995,134864.8803,103006.3006,176250.0535,129177.3132,141401.7825,157165.7016,357288.0338,114996.6181,101954.8059,238420.7098,99903.61916,136421.1845,204090.4306,130771.0780,134021.3278,127346.2300,237975.1356,114234.7199,140148.0011,153217.4086,111933.5443,120418.1744,108199.6723,114655.6553,67022.84798,59396.79654,75761.25105,74759.61748,78462.00717,59595.54049,109648.9517,122086.0211,139031.6994,106565.7660,2000
1,133722.0810,130752.3037,114828.7925,236089.3464,170977.4606,134701.3599,102946.9473,176242.8306,129255.0271,141433.9268,157078.6344,357668.9014,114967.1176,102105.5270,238672.3310,100150.70430,136408.0616,204338.8891,131298.6749,134244.8557,127383.7142,238281.5165,114257.3552,140366.4934,153159.5924,111796.6514,120563.6059,108282.8856,114860.6277,67028.59704,59551.03003,75926.07485,74640.54105,79355.73934,59567.71340,109740.4218,122300.8096,139180.9959,106579.0831,2000
2,133920.3002,130951.2645,114864.0270,236212.4905,171103.4612,134518.5218,103052.1521,176293.6291,129491.7089,141676.9367,157233.4277,357788.0513,115048.0005,102254.2967,238817.6176,100276.44360,136527.2451,204696.3115,131325.6071,134443.0426,127576.7276,238415.6158,114277.5831,140505.9568,153125.3794,111851.6178,120775.0762,108363.9959,115024.9297,67062.35524,59646.86405,76083.86365,74911.77298,79454.30914,59828.31922,109833.7113,122567.1789,139353.8737,106609.2298,2000
3,134421.5760,131295.6243,114834.3462,236675.9923,171505.2725,134202.7662,103213.1031,176309.3300,129892.5552,142145.8382,157633.7710,358227.8562,115255.9695,102591.2276,239630.4138,100519.50580,136751.2533,205249.2089,131643.8310,134938.3835,127918.9209,238841.0363,114224.7793,140811.5359,153064.3536,111906.3835,121119.4287,108578.5558,115366.3266,67056.51449,59908.55684,76481.87708,75462.48096,79845.05303,60303.02390,110058.1585,123138.4887,139741.4130,106595.3302,2000
4,134902.4056,131830.7726,114879.1655,237166.4369,171891.9131,134224.9822,103522.6453,176380.3570,130431.3599,142857.7981,158428.7234,358088.2233,115561.9971,102882.6121,240676.0497,100588.10170,137143.2235,205689.9952,132048.5310,135405.2776,128418.6830,239173.3826,114230.7856,141157.3413,153151.4427,112117.2720,121289.2496,108888.1967,115611.4614,67063.18552,60064.29336,76844.56167,76526.86530,79747.66564,60898.13109,110317.0658,123795.9979,140167.0446,106646.0031,2000


In [12]:
#rename geos with module
data = data.rename(columns = tofullcensus)

In [13]:
#group by the average of the year, transpose, make the column a region name, then go from wide to long with .melt()
data = data.groupby(['Year']).mean()
data = data.transpose().reset_index()
data = data.rename(columns = {'RegionName':'NAME'})
data = data.set_index('NAME')
data = data.melt(value_name = 'Home Value', ignore_index = False)
data = data.reset_index()

In [14]:
#make sure year is formatted as an integer
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [15]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Home Value                                                                                            \
Year                                   2000           2001           2002           2003           2004           2005           2006   
NAME                                                                                                                                    
Allen County, Kentucky         60256.966088   62785.270424   64029.063328   65447.719088   68034.257848   71163.803689   73333.080850   
Ashland City town, Tennessee  101175.202872  106349.054658  109715.663117  112660.693750  114488.813208  118562.107667  124332.081608   

                                                                                                                                       \
Year                                   2007           2008           2009           2010           2011           2012           2013   
NAME                                                                                                                                    
Allen County, Kentucky         75086.232972   75267.916787   75364.410652   75403.022865   74957.241032   77827.126945   78864.069987   
Ashland City town, Tennessee  131865.367842  135135.641200  132180.914017  128221.559700  124465.573517  124912.417333  128082.086967   

                                                                                                                                                    \
Year                                   2014           2015           2016           2017         2018           2019           2020           2021   
NAME                                                                                                                                                 
Allen County, Kentucky         82314.703165   84305.494823   90369.181467   97708.527660  108755.1762  113733.550817  124021.437133  141112.155017   
Ashland City town, Tennessee  132615.223050  143163.879242  158616.478942  176281.635075  190026.7551  206590.078833  225606.549925  264524.085242   

                                                                           
Year                                   2022           2023           2024  
NAME                                                                       
Allen County, Kentucky        160010.993867  166817.968408  173006.581678  
Ashland City town, Tennessee  310663.552633  314632.324333  329356.863844

In [16]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

Home Value                                                                                            \
                                       2000           2001           2002           2003           2004           2005           2006   
                                       None           None           None           None           None           None           None   
NAME                                                                                                                                    
Allen County, Kentucky         60256.966088   62785.270424   64029.063328   65447.719088   68034.257848   71163.803689   73333.080850   
Ashland City town, Tennessee  101175.202872  106349.054658  109715.663117  112660.693750  114488.813208  118562.107667  124332.081608   
Brentwood city, Tennessee     359034.423183  365395.737725  371277.112942  382878.273358  402798.705958  436804.033208  492497.848692   

                                                                                                                                       \
                                       2007           2008           2009           2010           2011           2012           2013   
                                       None           None           None           None           None           None           None   
NAME                                                                                                                                    
Allen County, Kentucky         75086.232972   75267.916787   75364.410652   75403.022865   74957.241032   77827.126945   78864.069987   
Ashland City town, Tennessee  131865.367842  135135.641200  132180.914017  128221.559700  124465.573517  124912.417333  128082.086967   
Brentwood city, Tennessee     539090.832150  528482.553808  499454.797075  480584.025817  483393.491567  488896.802725  518389.815050   

                                                                                                                                       \
                                       2014           2015           2016           2017           2018           2019           2020   
                                       None           None           None           None           None           None           None   
NAME                                                                                                                                    
Allen County, Kentucky         82314.703165   84305.494823   90369.181467   97708.527660  108755.176200  113733.550817  124021.437133   
Ashland City town, Tennessee  132615.223050  143163.879242  158616.478942  176281.635075  190026.755100  206590.078833  225606.549925   
Brentwood city, Tennessee     553124.602200  585137.648133  620344.710158  652680.895350  681670.074408  728462.185433  766914.267383   

                                                                                       
                                       2021          2022          2023          2024  
                                       None          None          None          None  
NAME                                                                                   
Allen County, Kentucky        141112.155017  1.600110e+05  1.668180e+05  1.730066e+05  
Ashland City town, Tennessee  264524.085242  3.106636e+05  3.146323e+05  3.293569e+05  
Brentwood city, Tennessee     939835.043192  1.218505e+06  1.196684e+06  1.241024e+06

In [17]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = df_pivot.columns.get_level_values(0).unique().tolist()
first_level

['Home Value']

In [18]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [19]:
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [20]:
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'Zillow'

In [21]:
#final check
data.head()

,NAME,Year,Time Frame,Home Value,Home Value % Change,Home Value Change,GEO_ID,Source
0,"Allen County, Kentucky",2000,None,60256.966088,NaN,NaN,0500000US21003,Zillow
1,"Allen County, Kentucky",2001,None,62785.270424,NaN,NaN,0500000US21003,Zillow
2,"Allen County, Kentucky",2002,None,64029.063328,NaN,NaN,0500000US21003,Zillow
3,"Allen County, Kentucky",2003,None,65447.719088,NaN,NaN,0500000US21003,Zillow
4,"Allen County, Kentucky",2004,None,68034.257848,NaN,NaN,0500000US21003,Zillow


In [22]:
data['NAME'].unique()

array(['Allen County, Kentucky', 'Ashland City town, Tennessee',
       'Brentwood city, Tennessee', 'Cheatham County, Tennessee',
       'Clarksville city, Tennessee', 'Columbia city, Tennessee',
       'Davidson County, Tennessee', 'Dickson County, Tennessee',
       'Fairview city, Tennessee', 'Franklin city, Kentucky',
       'Gallatin city, Tennessee', 'Hendersonville city, Tennessee',
       'Houston County, Tennessee', 'Humphreys County, Tennessee',
       'La Vergne city, Tennessee', 'Lebanon city, Tennessee',
       'Macon County, Tennessee', 'Maury County, Tennessee',
       'Montgomery County, Tennessee', 'Mount Juliet city, Tennessee',
       'Murfreesboro city, Tennessee',
       'Nashville-Davidson metropolitan government (balance), Tennessee',
       'Nolensville town, Tennessee', 'Robertson County, Tennessee',
       'Rutherford County, Tennessee', 'Smyrna town, Tennessee',
       'Spring Hill city, Tennessee', 'Springfield city, Tennessee',
       'Stewart County, Tenn

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12025 entries, 0 to 12024
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NAME                 12025 non-null  object 
 1   Year                 12025 non-null  object 
 2   Time Frame           12025 non-null  object 
 3   Home Value           925 non-null    float64
 4   Home Value % Change  11100 non-null  float64
 5   Home Value Change    11100 non-null  float64
 6   GEO_ID               11375 non-null  object 
 7   Source               12025 non-null  object 
dtypes: float64(3), object(5)
memory usage: 751.7+ KB


In [25]:
#export to the SQLite database as Zillow annual data
conn = sq.connect('../Outputs/Zillow.db')
data.to_sql('Annual_HomeValue', conn, if_exists = 'replace', index = False)

12025